# Can we identify faulty or suboptimally performing equipment?

In [1]:
#setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

gd=pd.read_csv("Train_Plant1.csv")

In [2]:
#Add columns
gd['AC_power_mean_dif']=gd['AC_POWER']-gd['AC_POWER'].mean()
gd['DATE_TIME'] = pd.to_datetime(gd['DATE_TIME'], errors='coerce')
gd['date']=pd.to_datetime(gd['DATE_TIME'].dt.date)
gd['time']=gd['DATE_TIME'].dt.time



In [3]:
#Per inverter new columns
uurimiseks= gd.copy()
groups = dict(list(uurimiseks.groupby("SOURCE_KEY")))

shift_nr=3
shift_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION']

gr_nr=0

#inverter based shifting
for group in groups:
    gr_nr+=1
    for i in range(shift_nr):
        for j in range(len(shift_columns)):
            groups[group][shift_columns[j]+'_PER_INVERTER_'+str(i+1)]=groups[group][shift_columns[j]].shift(-i+1)

    groups[group]['INVERTER']=group
    groups[group]['AC_DIF_AVG_PER_INVERTER'] = np.mean(groups[group]['AC_POWER']) - groups[group]['AC_POWER']
    groups[group]['DC_DIF_AVG_PER_INVERTER'] = np.mean(groups[group]['DC_POWER']) - groups[group]['DC_POWER']
    
    if gr_nr==1:
        gen_data=pd.DataFrame(columns =groups[group].keys())
    
    #gen_data.append(groups[group], ignore_index=True) - did not seem to work and was replaced with concat
    gen_data=pd.concat([gen_data,groups[group]], ignore_index=True)
    
gen_data=gen_data.fillna(value=0)
gen_data

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,DC_POWER_PER_INVERTER_3,AC_POWER_PER_INVERTER_3,DAILY_YIELD_PER_INVERTER_3,TOTAL_YIELD_PER_INVERTER_3,AMBIENT_TEMPERATURE_PER_INVERTER_3,MODULE_TEMPERATURE_PER_INVERTER_3,IRRADIATION_PER_INVERTER_3,INVERTER,AC_DIF_AVG_PER_INVERTER,DC_DIF_AVG_PER_INVERTER
0,2020-05-15 00:00:00,1BY6WEcLGh8j5v7,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,0.0,0.0,6259559.0,25.084589,22.761668,0.0,1BY6WEcLGh8j5v7,197.340827,2017.664611
1,2020-05-15 00:15:00,1BY6WEcLGh8j5v7,25.084589,22.761668,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,0.0,0.0,6259559.0,24.935753,22.592306,0.0,1BY6WEcLGh8j5v7,197.340827,2017.664611
2,2020-05-15 00:30:00,1BY6WEcLGh8j5v7,24.935753,22.592306,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,0.0,0.0,6259559.0,24.846130,22.360852,0.0,1BY6WEcLGh8j5v7,197.340827,2017.664611
3,2020-05-15 00:45:00,1BY6WEcLGh8j5v7,24.846130,22.360852,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,0.0,0.0,6259559.0,24.621525,22.165423,0.0,1BY6WEcLGh8j5v7,197.340827,2017.664611
4,2020-05-15 01:00:00,1BY6WEcLGh8j5v7,24.621525,22.165423,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,0.0,0.0,6259559.0,24.536092,21.968571,0.0,1BY6WEcLGh8j5v7,197.340827,2017.664611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53851,2020-06-09 10:45:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,zVJPv84UY57bAof,210.659192,2154.572792
53852,2020-06-09 11:00:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,zVJPv84UY57bAof,210.659192,2154.572792
53853,2020-06-09 11:15:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,zVJPv84UY57bAof,210.659192,2154.572792
53854,2020-06-09 11:30:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,zVJPv84UY57bAof,210.659192,2154.572792


In [4]:
groups[group].keys()

Index(['DATE_TIME', 'SOURCE_KEY', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE',
       'IRRADIATION', 'PLANT_ID', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD',
       'TOTAL_YIELD', 'ARTIFICIAL', 'AC_power_mean_dif', 'date', 'time',
       'DC_POWER_PER_INVERTER_1', 'AC_POWER_PER_INVERTER_1',
       'DAILY_YIELD_PER_INVERTER_1', 'TOTAL_YIELD_PER_INVERTER_1',
       'AMBIENT_TEMPERATURE_PER_INVERTER_1',
       'MODULE_TEMPERATURE_PER_INVERTER_1', 'IRRADIATION_PER_INVERTER_1',
       'DC_POWER_PER_INVERTER_2', 'AC_POWER_PER_INVERTER_2',
       'DAILY_YIELD_PER_INVERTER_2', 'TOTAL_YIELD_PER_INVERTER_2',
       'AMBIENT_TEMPERATURE_PER_INVERTER_2',
       'MODULE_TEMPERATURE_PER_INVERTER_2', 'IRRADIATION_PER_INVERTER_2',
       'DC_POWER_PER_INVERTER_3', 'AC_POWER_PER_INVERTER_3',
       'DAILY_YIELD_PER_INVERTER_3', 'TOTAL_YIELD_PER_INVERTER_3',
       'AMBIENT_TEMPERATURE_PER_INVERTER_3',
       'MODULE_TEMPERATURE_PER_INVERTER_3', 'IRRADIATION_PER_INVERTER_3',
       'INVERTER', 'AC_DIF_AVG_PER

In [8]:
#Group by day labels
'''
* label "FAULTY_according_to_DAILY_YIELD" - yield stays the same, while other inverters still increases
* label "INCORRECT_daily_yield" - yield per day decreases
* columns per inverter day average differences (total are done)
'''


uurimiseks= gen_data.copy()
groups = dict(list(uurimiseks.groupby("date")))

shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'AC_DIF_AVG_PER_DAY','DC_DIF_AVG_PER_DAY']

gr_nr=0

#inverter based shifting
for group in groups:
    #groups[group]['DAILY_YIELD_ERROR']=group
    
    groups[group]['AC_DIF_AVG_PER_DAY'] = np.mean(groups[group]['AC_POWER']) - groups[group]['AC_POWER']
    groups[group]['DC_DIF_AVG_PER_DAY'] = np.mean(groups[group]['DC_POWER']) - groups[group]['DC_POWER']
    
    
    gr_nr+=1
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_DAY_SHIFT_'+str(i+1)]=groups[group][production_columns[j]].shift(-i+1)


    if gr_nr==1:
        gen_data2=pd.DataFrame(columns =groups[group].keys())
    
    #gen_data.append(groups[group], ignore_index=True) - did not seem to work and was replaced with concat
    gen_data2=pd.concat([gen_data2,groups[group]], ignore_index=True)
    
gen_data2=gen_data2.fillna(value=0)
gen_data2

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,DAILY_YIELD_PER_DAY_SHIFT_2,TOTAL_YIELD_PER_DAY_SHIFT_2,AC_DIF_AVG_PER_DAY_PER_DAY_SHIFT_2,DC_DIF_AVG_PER_DAY_PER_DAY_SHIFT_2,DC_POWER_PER_DAY_SHIFT_3,AC_POWER_PER_DAY_SHIFT_3,DAILY_YIELD_PER_DAY_SHIFT_3,TOTAL_YIELD_PER_DAY_SHIFT_3,AC_DIF_AVG_PER_DAY_PER_DAY_SHIFT_3,DC_DIF_AVG_PER_DAY_PER_DAY_SHIFT_3
0,2020-05-15 00:00:00,1BY6WEcLGh8j5v7,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,6259559.0,260.810113,2664.412473,0.0,0.0,0.0,6259559.0,260.810113,2664.412473
1,2020-05-15 00:15:00,1BY6WEcLGh8j5v7,25.084589,22.761668,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,6259559.0,260.810113,2664.412473,0.0,0.0,0.0,6259559.0,260.810113,2664.412473
2,2020-05-15 00:30:00,1BY6WEcLGh8j5v7,24.935753,22.592306,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,6259559.0,260.810113,2664.412473,0.0,0.0,0.0,6259559.0,260.810113,2664.412473
3,2020-05-15 00:45:00,1BY6WEcLGh8j5v7,24.846130,22.360852,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,6259559.0,260.810113,2664.412473,0.0,0.0,0.0,6259559.0,260.810113,2664.412473
4,2020-05-15 01:00:00,1BY6WEcLGh8j5v7,24.621525,22.165423,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,6259559.0,260.810113,2664.412473,0.0,0.0,0.0,6259559.0,260.810113,2664.412473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53851,2020-06-09 10:45:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
53852,2020-06-09 11:00:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
53853,2020-06-09 11:15:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
53854,2020-06-09 11:30:00,zVJPv84UY57bAof,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000


In [9]:
groups[group].keys()

Index(['DATE_TIME', 'SOURCE_KEY', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE',
       'IRRADIATION', 'PLANT_ID', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD',
       'TOTAL_YIELD', 'ARTIFICIAL', 'AC_power_mean_dif', 'date', 'time',
       'DC_POWER_PER_INVERTER_1', 'AC_POWER_PER_INVERTER_1',
       'DAILY_YIELD_PER_INVERTER_1', 'TOTAL_YIELD_PER_INVERTER_1',
       'AMBIENT_TEMPERATURE_PER_INVERTER_1',
       'MODULE_TEMPERATURE_PER_INVERTER_1', 'IRRADIATION_PER_INVERTER_1',
       'DC_POWER_PER_INVERTER_2', 'AC_POWER_PER_INVERTER_2',
       'DAILY_YIELD_PER_INVERTER_2', 'TOTAL_YIELD_PER_INVERTER_2',
       'AMBIENT_TEMPERATURE_PER_INVERTER_2',
       'MODULE_TEMPERATURE_PER_INVERTER_2', 'IRRADIATION_PER_INVERTER_2',
       'DC_POWER_PER_INVERTER_3', 'AC_POWER_PER_INVERTER_3',
       'DAILY_YIELD_PER_INVERTER_3', 'TOTAL_YIELD_PER_INVERTER_3',
       'AMBIENT_TEMPERATURE_PER_INVERTER_3',
       'MODULE_TEMPERATURE_PER_INVERTER_3', 'IRRADIATION_PER_INVERTER_3',
       'INVERTER', 'AC_DIF_AVG_PER

In [10]:
# add inverter difference from average per that moment
'''
* columns "MOMENT_AVERAGE" - average value per this moment of inverter production info
* columns "DIF_FROM_AVERAGE" - inverter avg values in comparison with other inverters per this moment

'''


uurimiseks= gen_data2.copy()
groups = dict(list(uurimiseks.groupby("DATE_TIME")))

shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'AC_DIF_AVG_PER_DAY','DC_DIF_AVG_PER_DAY']

gr_nr=0

#inverter based shifting
for group in groups:
    #groups[group]['DAILY_YIELD_ERROR']=group
    
    groups[group]['AC_DIF_AVG_PER_MOMENT'] = np.mean(groups[group]['AC_POWER']) - groups[group]['AC_POWER']
    groups[group]['DC_DIF_AVG_PER_MOMENT'] = np.mean(groups[group]['DC_POWER']) - groups[group]['DC_POWER']
    
    
    gr_nr+=1
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_MOMENT_'+str(i+1)]=groups[group][production_columns[j]].shift(-i+1)


    if gr_nr==1:
        gen_data3=pd.DataFrame(columns =groups[group].keys())
    
    #gen_data.append(groups[group], ignore_index=True) - did not seem to work and was replaced with concat
    gen_data3=pd.concat([gen_data3,groups[group]], ignore_index=True)
    
gen_data3=gen_data3.fillna(value=0)
gen_data3

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,DAILY_YIELD_PER_MOMENT_2,TOTAL_YIELD_PER_MOMENT_2,AC_DIF_AVG_PER_DAY_PER_MOMENT_2,DC_DIF_AVG_PER_DAY_PER_MOMENT_2,DC_POWER_PER_MOMENT_3,AC_POWER_PER_MOMENT_3,DAILY_YIELD_PER_MOMENT_3,TOTAL_YIELD_PER_MOMENT_3,AC_DIF_AVG_PER_DAY_PER_MOMENT_3,DC_DIF_AVG_PER_DAY_PER_MOMENT_3
0,2020-05-15 00:00:00,1BY6WEcLGh8j5v7,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,6259559.0,...,0.0,6259559.0,260.810113,2664.412473,0.0,0.0,0.0,6183645.0,260.810113,2664.412473
1,2020-05-15 00:00:00,1IF53ai7Xc0U56Y,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,6183645.0,...,0.0,6183645.0,260.810113,2664.412473,0.0,0.0,0.0,6987759.0,260.810113,2664.412473
2,2020-05-15 00:00:00,3PZuoBAID5Wc2HD,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,6987759.0,...,0.0,6987759.0,260.810113,2664.412473,0.0,0.0,0.0,7602960.0,260.810113,2664.412473
3,2020-05-15 00:00:00,7JYdWkrLSPkdwr4,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,7602960.0,...,0.0,7602960.0,260.810113,2664.412473,0.0,0.0,0.0,7158964.0,260.810113,2664.412473
4,2020-05-15 00:00:00,McdE0feGgRqW7Ca,25.184316,22.857507,0.0,4135001.0,0.0,0.0,0.0,7158964.0,...,0.0,7158964.0,260.810113,2664.412473,0.0,0.0,0.0,7206408.0,260.810113,2664.412473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53851,2020-06-09 11:45:00,uHbuxQJl8lW7ozc,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
53852,2020-06-09 11:45:00,wCURE6d3bPkepu2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
53853,2020-06-09 11:45:00,z9Y9gH1T5YWrNuG,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
53854,2020-06-09 11:45:00,zBIq5rxdHJRwDNY,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000


In [ ]:
#other labels adding
'''
* label "INCORRECT_total_yield" - yield per period decreases
* label "INCORRECT_according_to_DC_vs_AC" - DC and AC ratio is off
* label "INCORRECT_according_to_DC_AC_DAILY_YIELD" - DC and AC increases, while DAILY_YIELD does not
'''